In [42]:
import natsort
import collections
import pandas as pd
from itertools import product

In [8]:
ports = "pa0 pa1 pa4 pb0 pc1 pc0 pb8 pb9 pa5 pa6 pa7 pb6 pc7 pa9 pa8 pb10 pb4 pb5 pb3 pa10 pa2 pa3".split(" ")
pins_ard = "a0 a1 a2 a3 a4 a5 d15 d14 d13 d12 d11 d10 d9 d8 d7 d6 d5 d4 d3 d2 d1 d0".split(" ")

In [193]:
port_pin_map = dict(zip(ports, pins_ard))
pin_port_map = dict(zip(pins_ard, ports))

In [12]:
timers = "pa0 2/1 5/1, pa1 2/2 5/2, pb0 1/2n 8/2n 3/3, pb8 10/1 2/1 4/3, pb9 11/1 2/2 4/4, pa5 2/1 8/1n, \
pa6 13/1 3/1, pa7 14/1 3/2 8/1, pb6 4/1, pc7 3/2 8/2, pa9 1/2, pa8 1/1, pb10 2/3, pb4 3/1, pb5 3/2, pa10 1/3".split(",")

In [29]:
def parse(t):
    t = t.strip().split(" ")
    return port_pin_map[t[0]], t[1:]

In [30]:
pin_tim_map = {parse(t)[0]:parse(t)[1] for t in timers}

In [33]:
for k in sorted(pin_tim_map):
    print(k, pin_tim_map[k])

a0 ['2/1', '5/1']
a1 ['2/2', '5/2']
a3 ['1/2n', '8/2n', '3/3']
d10 ['4/1']
d11 ['14/1', '3/2', '8/1']
d12 ['13/1', '3/1']
d13 ['2/1', '8/1n']
d14 ['11/1', '2/2', '4/4']
d15 ['10/1', '2/1', '4/3']
d2 ['1/3']
d4 ['3/2']
d5 ['3/1']
d6 ['2/3']
d7 ['1/1']
d8 ['1/2']
d9 ['3/2', '8/2']


In [40]:
collections.OrderedDict(natsort.natsorted(pin_tim_map.items())).ma

OrderedDict([('a0', ['2/1', '5/1']),
             ('a1', ['2/2', '5/2']),
             ('a3', ['1/2n', '8/2n', '3/3']),
             ('d2', ['1/3']),
             ('d4', ['3/2']),
             ('d5', ['3/1']),
             ('d6', ['2/3']),
             ('d7', ['1/1']),
             ('d8', ['1/2']),
             ('d9', ['3/2', '8/2']),
             ('d10', ['4/1']),
             ('d11', ['14/1', '3/2', '8/1']),
             ('d12', ['13/1', '3/1']),
             ('d13', ['2/1', '8/1n']),
             ('d14', ['11/1', '2/2', '4/4']),
             ('d15', ['10/1', '2/1', '4/3'])])

In [161]:
t = [list(product([key], pin_tim_map[key])) for key in pin_tim_map]
flat_mapping = [item for sublist in t for item in sublist]

df_pin_tim = pd.DataFrame(flat_mapping, columns=("pin", "tim"))

df_pin_tim[['tim', 'chan']] = pd.DataFrame(df_pin_tim['tim'].apply(lambda x: x.split("/")).to_list())

df_pin_tim['pin'] = df_pin_tim['pin'].apply(lambda x: x.replace("a", "x"))

df_pin_tim['tim'] = df_pin_tim['tim'].astype(int)

In [168]:
data = df_pin_tim.groupby('tim')['pin'].apply(lambda x: list(natsort.natsorted(x))).to_dict()

list_tims = df_pin_tim.tim.unique()
list_tims.sort()

assigned = {}

def solve(tim_idx):
    tim = list_tims[tim_idx]
    this_pin_timers = data[tim]
    selected_pin = this_pin_timers[0]
    assigned[tim] = selected_pin
    for key in data:
        if selected_pin in data[key]:
            data[key].remove(selected_pin)
    if tim_idx < len(list_tims) - 1:
        solve(tim_idx + 1)
        
solve(0)
print(assigned)

{1: 'd2', 2: 'd6', 3: 'd4', 4: 'd10', 5: 'x0', 8: 'd9', 10: 'd15', 11: 'd14', 13: 'd12', 14: 'd11'}


In [205]:
res = []
for idx, (k, v) in enumerate(assigned.items()):
    res.append((df[(df.tim == k) & (df.pin == v)]).values[0])
    
res = pd.DataFrame(res, columns=['pin', 'tim', 'chan'])

res['pin'] = res['pin'].apply(lambda x: x.replace("x", "a"))
res['port'] = res['pin'].apply(lambda x: pin_port_map[x])

res = res.reset_index().rename(columns={'index': "channel"})

In [208]:
res.to_csv('channel_mapping.csv', index=None)

In [207]:
res

,channel,pin,tim,chan,port
0,0,d2,1,3,pa10
1,1,d6,2,3,pb10
2,2,d4,3,2,pb5
3,3,d10,4,1,pb6
4,4,a0,5,1,pa0
5,5,d9,8,2,pc7
6,6,d15,10,1,pb8
7,7,d14,11,1,pb9
8,8,d12,13,1,pa6
9,9,d11,14,1,pa7
